# SE-2200E Notebook 3: Data Transformation

Ningsong Shen

February 12, 2021

## Summary

We want to transform the data to make it easier to use, we want to select a subset of data to use, remove outliers, remove irrelevant features so that we can apply simple techniques. Professor Katchabaw suggested this method which would make testing and isolating any issues easier.

In [2]:
import pandas as pd 


In [3]:
df = pd.read_csv('../clean_data/prepared_data.csv')

C:\Users\nings\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df.head(50)

,trip_id,start_date,route_id,stop_sequence,realtime_departure_time,vehicle_id,scheduled_departure_time
0,1346803,20201104,24,41,1604508849,3140.0,11:53:34
1,1346803,20201105,24,41,1604595120,3177.0,11:53:34
2,1346803,20201106,24,41,1604681520,3143.0,11:53:34
3,1346803,20201109,24,41,1604940720,3144.0,11:53:34
4,1346803,20201110,24,41,1605027120,3178.0,11:53:34
5,1346803,20201111,24,41,1605113520,3503.0,11:53:34
6,1346803,20201112,24,41,1605199920,3510.0,11:53:34
7,1346803,20201113,24,41,1605286320,3338.0,11:53:34
8,1346803,20201116,24,41,1605545520,3164.0,11:53:34
9,1346803,20201117,24,41,1605631920,3520.0,11:53:34


Let's choose some features: a single route id and stop sequence will identify a single stop.

In [5]:
stop_route = df.loc[(df['trip_id'] == 1346803) & (df['stop_sequence'] == 41)]

In [7]:
stop_route.head(50)

,trip_id,start_date,route_id,stop_sequence,realtime_departure_time,vehicle_id,scheduled_departure_time
0,1346803,20201104,24,41,1604508849,3140.0,11:53:34
1,1346803,20201105,24,41,1604595120,3177.0,11:53:34
2,1346803,20201106,24,41,1604681520,3143.0,11:53:34
3,1346803,20201109,24,41,1604940720,3144.0,11:53:34
4,1346803,20201110,24,41,1605027120,3178.0,11:53:34
5,1346803,20201111,24,41,1605113520,3503.0,11:53:34
6,1346803,20201112,24,41,1605199920,3510.0,11:53:34
7,1346803,20201113,24,41,1605286320,3338.0,11:53:34
8,1346803,20201116,24,41,1605545520,3164.0,11:53:34
9,1346803,20201117,24,41,1605631920,3520.0,11:53:34
